# Promethee I Ranking

This module shows an example of how to use the "Promethee I Ranking" module

In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import sys
sys.path.append('../..')

## Definition of inputs and problem formalization

In [32]:
import pandas as pd
from modular_parts.preference import compute_preference_indices
from modular_parts.flows import calculate_promethee_outranking_flows, calculate_net_outranking_flows
from modular_parts.ranking import calculate_prometheeI_ranking
from core.enums import Direction, GeneralCriterion, FlowType


alternatives = [f"a{i}" for i in range(1, 9)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22],
                                          [10, 75, 20]], index=alternatives, columns=criteria)

## Required data

### Required prefereces

In [33]:
aggregated_alternatives_vs_alternatives_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                                    preference_thresholds,
                                                                                    indifference_thresholds,
                                                                                    standard_deviations,
                                                                                    generalised_criteria,
                                                                                    criteria_directions,
                                                                                    criteria_weights)
aggregated_alternatives_vs_alternatives_preferences

,a1,a2,a3,a4,a5,a6,a7,a8
a1,0.00,0.88,0.8,0.5,0.0,0.5,0.3,0.8
a2,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
a3,0.12,0.70,0.0,0.0,0.0,0.5,0.0,0.0
a4,0.50,1.00,0.5,0.0,0.0,0.5,0.3,0.5
a5,1.00,1.00,1.0,0.5,0.0,0.5,0.8,1.0
a6,0.50,0.50,0.5,0.5,0.5,0.0,0.5,0.5
a7,0.70,1.00,1.0,0.5,0.0,0.5,0.0,1.0
a8,0.12,0.70,0.0,0.0,0.0,0.5,0.0,0.0


### Required flows

In [34]:
prometheeI_flows = calculate_promethee_outranking_flows(aggregated_alternatives_vs_alternatives_preferences,
                                                        flow_type=FlowType.BASIC)
prometheeI_flows

,positive,negative
a1,0.540000,0.420000
a2,0.000000,0.825714
a3,0.188571,0.542857
a4,0.471429,0.285714
a5,0.828571,0.071429
a6,0.500000,0.428571
a7,0.671429,0.271429
a8,0.188571,0.542857


## Usage of Promethee I Ranking

### Weak preference

In [35]:
promethee_I_ranking_weak = calculate_prometheeI_ranking(prometheeI_flows, weak_preference=True)
promethee_I_ranking_weak

,a1,a2,a3,a4,a5,a6,a7,a8
a1,None,S,S,?,?,S,?,S
a2,?,None,?,?,?,?,?,?
a3,?,S,None,?,?,?,?,S
a4,?,S,S,None,?,?,?,S
a5,S,S,S,S,None,S,S,S
a6,?,S,S,?,?,None,?,S
a7,S,S,S,S,?,S,None,S
a8,?,S,S,?,?,?,?,None


### Strong preference

In [36]:
promethee_I_ranking_strong = calculate_prometheeI_ranking(prometheeI_flows, weak_preference=False)
promethee_I_ranking_strong

,a1,a2,a3,a4,a5,a6,a7,a8
a1,None,P,P,?,?,P,?,P
a2,?,None,?,?,?,?,?,?
a3,?,P,None,?,?,?,?,I
a4,?,P,P,None,?,?,?,P
a5,P,P,P,P,None,P,P,P
a6,?,P,P,?,?,None,?,P
a7,P,P,P,P,?,P,None,P
a8,?,P,I,?,?,?,?,None


The additional alternative **a8** differs from **a3** by 1 unit on the criterion **c1**, but since 1 unit of difference on this criterion still means indifference, these alternatives are in an indifference relation. (The result may be more spectacular for examples where the preference and indifference thresholds have higher values)